In [5]:
from langchain_community.llms import Ollama 
from langchain_ollama import OllamaLLM

llm__ollama = OllamaLLM(
    model="llama3.2"
)  # assuming you have Ollama installed and have llama3 model pulled with `ollama pull llama3 `

# llm__ollama.invoke("generate a sql query which can count the total rows")   

llm__ollama_ans = OllamaLLM(
    model="llama3.2"
)  # assuming you have Ollama installed and have llama3 model pulled with `ollama pull llama3 `


In [6]:
from langchain_community.utilities import SQLDatabase
import urllib.parse
# from sqlalchemy import create_engine
quoted_password = urllib.parse.quote("Prachi@28")

snowflake_db = SQLDatabase.from_uri('snowflake://{user}:{password}@{account_identifier}/SNOWFLAKE_SAMPLE_DATA/TPCDS_SF100TCL'.format(
        user='PRACHIBH',
        password=quoted_password,
        account_identifier='dmhwtcd-nt69450',
    ))

# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM Artist LIMIT 10;")

# construct sqlalchemy engine from uri 
# error : full path need to be mentioned => so, full path as in till the db with .db included ?  

/Users/prachi-bhardwaj/Downloads/llm/.venv/lib/python3.12/site-packages/snowflake/sqlalchemy/base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


In [7]:
from langchain_core.output_parsers.list import CommaSeparatedListOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.language_models import BaseLanguageModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import BasePromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain.chains.sql_database.prompt import PROMPT, SQL_PROMPTS

from typing import Any, Dict, List, Optional, Union, TypedDict, TYPE_CHECKING

if TYPE_CHECKING:
    from langchain_community.utilities.sql_database import SQLDatabase

def _strip(text: str) -> str:
    print("insisde _strip")
    return text.strip()

class SQLInput(TypedDict):
    """Input for a SQL Chain."""
    question: str

class SQLInputWithTables(TypedDict):
    """Input for a SQL Chain."""
    question: str
    table_names_to_use: List[str]

def create_sql_query_chain(
    llm: BaseLanguageModel,
    db: SQLDatabase,
    prompt: Optional[BasePromptTemplate] = None,
    k: int = 1,
) -> Runnable[Union[SQLInput, SQLInputWithTables, Dict[str, Any]], str]:

    if prompt is not None:
        prompt_to_use = prompt
    elif db.dialect in SQL_PROMPTS:
        prompt_to_use = SQL_PROMPTS[db.dialect]
    else:
        prompt_to_use = PROMPT

    if {"input", "top_k", "table_info"}.difference(
        prompt_to_use.input_variables + list(prompt_to_use.partial_variables)
    ):
        raise ValueError(
            f"Prompt must have input variables: 'input', 'top_k', "
            f"'table_info'. Received prompt with input variables: "
            f"{prompt_to_use.input_variables}. Full prompt:\n\n{prompt_to_use}"
        )
    if "dialect" in prompt_to_use.input_variables:
        prompt_to_use = prompt_to_use.partial(dialect=db.dialect)

    inputs = {
        "input": lambda x: x["question"] + "\nSQLQuery: ",
        "table_info": lambda x: db.get_table_info(
            table_names=x.get("table_names_to_use")
        ),
    }

    return (
        RunnablePassthrough.assign(**inputs)  # type: ignore
        | prompt_to_use.partial(top_k=str(k))
        | llm.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
        | _strip
    )

# Example usage:
# Make sure to replace `llm__ollama` and `db` with your actual language model and database objects.



In [10]:
import re 
from langchain_community.tools.sql_database.tool import BaseSQLDatabaseTool, _QuerySQLDataBaseToolInput, QuerySQLDataBaseTool

from typing import Any, Dict, Optional, Sequence, Type, Union

from sqlalchemy.engine import Result

from pydantic import BaseModel, Field, root_validator, model_validator, ConfigDict

from langchain_core.language_models import BaseLanguageModel
from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.tools import BaseTool
from langchain_community.tools.sql_database.prompt import QUERY_CHECKER 

class QuerySQLDataBaseToolCustom(QuerySQLDataBaseTool):
    def __init__(self, db: SQLDatabase):
        super().__init__(db=db)
        self.db = db  # Initialize the database connection

    def _run(
        self,
        response: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> Union[str, Sequence[Dict[str, Any]], Result]:
        """Custom run logic for executing the SQL query."""
        pattern = r"SQLQuery:\s*((?:.|\n)*?)(;|$)"
        match = re.search(pattern, response)
        if match:
            sql_query = match.group(1)
            # Execute the dynamically extracted SQL query 
            ans = self.db.run("SELECT cd_dep_employed_count FROM customer_demographics WHERE cd_gender = 'M' AND cd_credit_rating = 'Good' limit 1;") 
            print(ans)
            return ans, 
        else:
            return "SQL not found"


In [11]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query} 
SQL Result: {result}

Answer: """
) 

write_query = create_sql_query_chain(llm__ollama,snowflake_db)
execute_query = QuerySQLDataBaseToolCustom(snowflake_db)
answer = answer_prompt | llm__ollama_ans | StrOutputParser()
chain_3 = (
    RunnablePassthrough.assign(query=write_query).assign(
        result= itemgetter("query") | execute_query
    ) | answer
) 

chain_3.invoke({
    "question" : "extract male count from CUSTOMER_DEMOGRAPHICS where CD_CREDIT_RATING are good",
    "table_names_to_use": ['customer_demographics']
}) 


insisde _strip
[(0,)]


"The result indicates that there are no records in the CUSTOMER_DEMOGRAPHICS table where the CD_CREDIT_RATING is 'Good'. Therefore, the answer to the user's question is:\n\nThere is 1 male customer with a good credit rating."